In [ ]:
import numpy as np
import pandas as pd
import sklearn
import torch
import matplotlib.pyplot as plt

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
#nltk.download('stopwords')

#stop_words = set(stopwords.words('russian') + ['\n', '\r'])
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def preprocess(text):
    text = list(filter(str.isalpha, word_tokenize(text.lower())))
    text = list(lemmatizer.lemmatize(word) for word in text)
    #text = list(word for word in text if word not in stop_words)
    return ' '.join(text)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

from sklearn.decomposition import LatentDirichletAllocation
import collections, re
from sklearn.feature_extraction import DictVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
!pip install catboost

     |████████████████████████████████| 67.3MB 41kB/s 


In [ ]:
from catboost import CatBoostClassifier

In [ ]:
data = pd.read_csv('/content/pulse_stage1_patch.csv')

In [ ]:
data['text'] = data.apply(lambda row: preprocess(row.message), axis=1)

In [ ]:
data

,message,label,split,text
0,Пиздабол ещё тот,True,train,пиздабол ещё тот
1,"Заколебали кукарекать, скоро весь пульс будет ...",True,train,заколебали кукарекать скоро весь пульс будет э...
2,На деле собака сутулая,True,train,на деле собака сутулая
3,Пшнх от сюда тварь,True,train,пшнх от сюда тварь
4,"плати за мое молчание клоп , сутки молчу 200 б...",True,train,плати за мое молчание клоп сутки молчу баксов
...,...,...,...,...
34602,"Тут не то, что ТА не работает, здравый смысл о...",False,train,тут не то что та не работает здравый смысл отс...
34603,Я тоже в бакс по 30 верил\nКогда он уже 50 сто...,False,train,я тоже в бакс по верил когда он уже стоил
34604,"Ахаха, тока щас заметил что за это гавно комис...",False,train,ахаха тока щас заметил что за это гавно комисс...
34605,"Я с Теслой месяц мучился с плечом , в итоге вс...",False,train,я с теслой месяц мучился с плечом в итоге все ...


In [ ]:
data['label','text'].to_csv('start_data.csv', index=False)

KeyError: ignored

In [ ]:
picabu['text'] = data.apply(lambda row: preprocess(row.message), axis=1)

In [ ]:
picabu = pd.read_csv('/content/labeled.csv')

In [ ]:
picabu['text'] = data.apply(lambda row: preprocess(row.message), axis=1)

In [ ]:
ok = pd.read_csv('/content/Ok.csv', error_bad_lines=False, sep=';')

In [ ]:
neg = ok[ok['label']!='NORMAL']
pos = ok[ok['label']=='NORMAL']

In [ ]:
neg.ttext = picabu.apply(lambda row: preprocess(row.text), axis=1)
pos.ttext = picabu.apply(lambda row: preprocess(row.text), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [ ]:
train = data[data['split']=='train']
test = data[data['split']=='valid']

In [ ]:
X_train = train.text
y_train = train.label
X_test = test.text
y_test = test.label

In [ ]:
X_train = pd.concat((train.text,picabu.text,neg.ttext, pos.ttext),axis=0, ignore_index=True)
y_train = pd.concat((train.label,picabu.toxic.astype(int), pd.Series(np.ones(neg.ttext.shape[0])), 
                     pd.Series(np.zeros(pos.ttext.shape[0]))),axis=0, ignore_index=True)
X_test = test.text
y_test = test.label

In [ ]:
train = {'title': y_train, 'text': X_train}
test = {'title': y_test, 'text': X_test}

In [ ]:
train = pd.DataFrame(train)
test = pd.DataFrame(test)

In [ ]:
np.sum(train.title.isna())

0

In [ ]:
train.to_csv('train_start.csv', index=False)
test.to_csv('test_start.csv', index=False)

In [ ]:
train = pd.read_csv('train_new.csv').dropna().reset_index()
test = pd.read_csv('test_new.csv').dropna().reset_index()

In [ ]:
np.sum(train.isna())

title    0
text     0
dtype: int64

In [ ]:
X_train.to_csv('X_train.csv')
X_test.to_csv('X_test.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')

In [ ]:
l = pd.read_csv('X_train')

In [ ]:
vectorizer = TfidfVectorizer(max_features=10000)
tfidf_train = np.asarray(vectorizer.fit_transform(X_train).todense())
tfidf_test = np.asarray(vectorizer.transform(X_test).todense())

ValueError: ignored

In [ ]:
tfidf_train.shape

(27701, 10000)

In [ ]:
tfidf_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
cat = CatBoostClassifier()

In [ ]:
cat.fit(tfidf_train, y_train)

Learning rate set to 0.042548
0:	learn: 0.6284131	total: 400ms	remaining: 6m 40s
1:	learn: 0.5708967	total: 698ms	remaining: 5m 48s
2:	learn: 0.5206829	total: 1.02s	remaining: 5m 39s
3:	learn: 0.4765130	total: 1.32s	remaining: 5m 28s
4:	learn: 0.4376003	total: 1.62s	remaining: 5m 23s
5:	learn: 0.4033723	total: 1.93s	remaining: 5m 19s
6:	learn: 0.3730163	total: 2.24s	remaining: 5m 17s
7:	learn: 0.3467681	total: 2.54s	remaining: 5m 14s
8:	learn: 0.3230017	total: 2.84s	remaining: 5m 12s
9:	learn: 0.3022207	total: 3.15s	remaining: 5m 11s
10:	learn: 0.2836752	total: 3.45s	remaining: 5m 10s
11:	learn: 0.2681328	total: 3.75s	remaining: 5m 8s
12:	learn: 0.2538347	total: 4.06s	remaining: 5m 8s
13:	learn: 0.2415396	total: 4.37s	remaining: 5m 7s
14:	learn: 0.2306811	total: 4.67s	remaining: 5m 6s
15:	learn: 0.2213149	total: 4.99s	remaining: 5m 6s
16:	learn: 0.2121885	total: 5.29s	remaining: 5m 6s
17:	learn: 0.2045102	total: 5.59s	remaining: 5m 5s
18:	learn: 0.1972802	total: 5.89s	remaining: 5m 4s


In [ ]:

pred = cat.predict(tfidf_test)
print('F1 on cat:', f1_score(pred, y_test))

F1 on cat: 0.3309352517985611


In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as download_api
model = Word2Vec(sentences=data.message, size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

KeyboardInterrupt: ignored

In [ ]:
corpus = download_api.load('word2vec-ruscorpora-300')

[==------------------------------------------------] 4.3% 8.5/198.8MB downloaded

KeyboardInterrupt: ignored

In [ ]:
russian_model = Word2Vec(corpus) 

TypeError: ignored

In [ ]:
! pip install navec

In [ ]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2021-04-14 12:16:03--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar.2’

navec_hudlit_v1_12B 100%[===================>]  50.56M  16.2MB/s    in 3.1s    

2021-04-14 12:16:07 (16.2 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar.2’ saved [53012480/53012480]



In [ ]:
from navec import Navec

path = '/content/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [ ]:
!pip install slovnet

In [ ]:
from slovnet.model.emb import NavecEmbedding

emb = NavecEmbedding(navec)
input = torch.tensor([497765])
output = emb(input)

/usr/local/lib/python3.7/dist-packages/slovnet/model/emb.py:46: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  torch.from_numpy(navec.pq.indexes),


In [ ]:
output.shape

torch.Size([1, 300])

In [ ]:
X_train_w2v = np.zeros((X_train.shape[0],300))
for idx, i in enumerate(X_train):
  vector = np.zeros((300))
  words_cnt = 1e-05
  try:
    for word in i:
      try:
        vector+=navec.get(word)
        words_cnt+=1
      except:
        continue
  except:
    try:
        vector+=navec.get(i)
        words_cnt+=1
    except:
        continue
  vector/=words_cnt
  X_train_w2v[idx]=vector

In [ ]:
X_test_w2v = np.zeros((X_test.shape[0],300))
for idx, i in enumerate(X_test):
  vector = np.zeros((300))
  words_cnt = 1e-05
  try:
    for word in i:
      try:
        vector+=navec.get(word)
        words_cnt+=1
      except:
        continue
  except:
    try:
        vector+=navec.get(i)
        words_cnt+=1
    except:
        continue
  vector/=words_cnt
  X_test_w2v[idx]=vector

In [ ]:
y_train.describe()

count    70937.000000
mean         0.284069
std          0.450973
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
dtype: float64

In [ ]:
cat_w2v = CatBoostClassifier()
cat_w2v.fit(X_train_w2v,y_train)

Learning rate set to 0.063571
0:	learn: 0.6807478	total: 267ms	remaining: 4m 27s
1:	learn: 0.6698805	total: 430ms	remaining: 3m 34s
2:	learn: 0.6604507	total: 610ms	remaining: 3m 22s
3:	learn: 0.6521987	total: 790ms	remaining: 3m 16s
4:	learn: 0.6450003	total: 954ms	remaining: 3m 9s
5:	learn: 0.6387655	total: 1.13s	remaining: 3m 7s
6:	learn: 0.6332774	total: 1.29s	remaining: 3m 3s
7:	learn: 0.6284480	total: 1.46s	remaining: 3m
8:	learn: 0.6242237	total: 1.62s	remaining: 2m 58s
9:	learn: 0.6205071	total: 1.78s	remaining: 2m 56s
10:	learn: 0.6173012	total: 1.94s	remaining: 2m 54s
11:	learn: 0.6144991	total: 2.11s	remaining: 2m 53s
12:	learn: 0.6120393	total: 2.27s	remaining: 2m 52s
13:	learn: 0.6098806	total: 2.45s	remaining: 2m 52s
14:	learn: 0.6078891	total: 2.65s	remaining: 2m 53s
15:	learn: 0.6061747	total: 2.81s	remaining: 2m 52s
16:	learn: 0.6046773	total: 2.98s	remaining: 2m 52s
17:	learn: 0.6033890	total: 3.15s	remaining: 2m 52s
18:	learn: 0.6022489	total: 3.31s	remaining: 2m 51s

In [ ]:
pred = cat_w2v.predict(X_test_w2v)
print('F1 on cat:', f1_score(pred, y_test))

F1 on cat: 0.2514285714285714


In [ ]:
for i in navec.vocab:
  print(i)
  break

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
output

tensor([[-3.4993e-01, -3.0756e-01, -2.7500e-01,  1.1932e-03, -1.7987e-01,
         -1.3807e-01,  1.0722e-02,  3.0190e-01, -9.9065e-02, -1.4211e-01,
         -6.3561e-01, -1.4722e-01, -3.9479e-01, -3.7867e-01, -7.1260e-02,
          2.0672e-01,  1.2577e-01, -3.2960e-01,  3.5796e-01,  2.7854e-01,
         -4.3397e-01, -1.4973e-02, -1.9574e-01,  3.1353e-01, -2.2685e-01,
          1.9929e-02,  4.0099e-01,  1.3460e-01,  7.5548e-03,  1.5212e-01,
         -2.1347e-01, -1.2314e-01,  4.7164e-01,  2.5409e-01,  4.4845e-01,
          1.9260e-01, -2.7073e-01, -2.0794e-01,  7.3013e-01,  1.0160e-01,
         -2.6584e-01, -3.6435e-01,  7.3061e-02, -3.9773e-01, -1.0634e-01,
         -1.9774e-01, -4.3082e-02,  2.9453e-01, -6.7992e-01, -4.7794e-01,
         -3.0099e-01, -5.4821e-01,  7.8595e-02,  2.4260e-01, -3.3104e-01,
         -9.4799e-03,  4.5769e-02,  3.2028e-01,  2.1499e-01,  2.7773e-01,
          1.8513e-01, -3.6358e-01, -1.5184e-01, -1.3534e-01, -4.6190e-05,
          9.6920e-02, -6.7579e-02,  3.

In [ ]:
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from collections import Counter

In [ ]:
vocab = {}
n=0
for i in train['text']:
    i = i.split(sep=' ')
    try:
      for word in i:
        if word not in vocab.keys():
          vocab[word]=n
          n+=1
    except:
      try:
        if i not in vocab.keys():
          vocab[i]=n
          n+=1
      except:
        continue

for i in test['text']:
    i = i.split(sep=' ')
    try:
      for word in i:
        if word not in vocab.keys():
          vocab[word]=n
          n+=1
    except:
      try:
        if i not in vocab.keys():
          vocab[i]=n
          n+=1
      except:
        continue

In [ ]:
len(vocab)

47592

In [ ]:
class Text_Data(Dataset):
    def __init__(self, data, vocab, max_len):
        self.vocab = vocab
        self.X = data.text
        self.y = data.title
        self.max_len = max_len

        

    def __getitem__(self, index):

        X = np.asarray([vocab[i] for i in self.X[index].split(sep=' ')])
        if len(X)>self.max_len:
            X=X[:self.max_len]
        elif len(X)<self.max_len:
            X=np.hstack((X, np.zeros(self.max_len)))
        return (torch.tensor(X), torch.tensor(self.y[index]))

    def __len__(self):
        return len(self.y)

In [ ]:
len(train.title)

70918

In [ ]:
max_len = 20
train_set = Text_Data(train, vocab, max_len)
test_set = Text_Data(test, vocab, max_len)

In [ ]:
train_set[0]

(tensor([0., 1., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.], dtype=torch.float64),
 tensor(1., dtype=torch.float64))

In [ ]:
for i in train_set:
  print(i)

Выходные данные были обрезаны до нескольких последних строк (5000).
        16070,   111,  1465,  8163,  1143, 12351, 18944,   284,  3743, 12581]), tensor(0., dtype=torch.float64))
(tensor([14083,   495, 17348,   153,    48,    12,   131,  1584,    90,    54,
         1084,   926,  2758, 25433,  7463,    92,  2948,    64,   524,    97]), tensor(0., dtype=torch.float64))
(tensor([5.5300e+02, 8.0000e+00, 4.7394e+04, 4.7395e+04, 2.0700e+02, 4.7600e+02,
        2.9500e+02, 1.5300e+02, 8.1000e+01, 8.5300e+02, 2.9110e+03, 4.7396e+04,
        6.7600e+02, 7.0700e+02, 7.1760e+03, 1.1810e+03, 4.7084e+04, 1.8713e+04,
        9.0000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00], dtype=torch.float64), tensor(0., dtype=torch.float64))
(tensor([  92.,  448.,   81., 3219.,  553., 79

KeyError: ignored

In [ ]:
train_dataloader = DataLoader(train_set, batch_size=16)
test_dataloader = DataLoader(test_set, batch_size=32,
    num_workers=1,
    pin_memory=True,
    shuffle=True)

In [ ]:
for i in train_dataloader:
  print(i)
  break

RuntimeError: ignored

In [ ]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [ ]:
class LSTM(nn.Module):

    def __init__(self, hidden=128):
        super(LSTM, self).__init__()

        self.embedding = nn.Embedding(len(vocab.keys()), 300)
        self.hidden = hidden
        self.lstm = nn.LSTM(input_size=300,
                            hidden_size=hidden,
                            num_layers=1,
                            batch_first=True,
                            bidirectional=True)
        self.drop = nn.Dropout(p=0.5)

        self.fc = nn.Linear(2*hidden, 1)

    def forward(self, text, text_len):

        text_emb = self.embedding(text)

        packed_input = pack_padded_sequence(text_emb, text_len, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.lstm(packed_input)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)

        out_forward = output[range(len(output)), text_len - 1, :self.hidden]
        out_reverse = output[:, 0, self.hidden:]
        out_reduced = torch.cat((out_forward, out_reverse), 1)
        text_fea = self.drop(out_reduced)

        text_fea = self.fc(text_fea)
        text_fea = torch.squeeze(text_fea, 1)
        text_out = torch.sigmoid(text_fea)

        return text_out

In [ ]:
def train()

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.fc = nn.Linear(embed_dim, num_class)
        '''self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()'''
        
    def forward(self, text):
        embedded = self.embedding(text)
        return self.fc(embedded)

In [ ]:
device = 'cuda:0'

In [ ]:
VOCAB_SIZE = len(vocab.keys())
EMBED_DIM = 32
NUN_CLASS = 2
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

In [ ]:
from torch.utils.data import DataLoader

def train_func(data, criterion, optimizer):

    # Train the model
    train_loss = 0
    train_acc = 0

    for i, (X, y) in enumerate(data):
        optimizer.zero_grad()
        X, y = X.to(device), y.to(device)
        output = model(text)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    #scheduler.step()
    
    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [ ]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_set, [train_len, len(train_set) - train_len])

for epoch in range(N_EPOCHS):

    #start_time = time.time()
    train_loss, train_acc = train_func(train_dataloader, criterion, optimizer)
    #valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

KeyError: ignored

In [ ]:
for i in train_dataloader:
    print(i)

KeyError: ignored

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time

import torch
from torchtext import data
import torch.nn as nn

In [ ]:
# source : https://gist.github.com/lextoumbourou/8f90313cbc3598ffbabeeaa1741a11c8
# to use DataFrame as a Data source

class DataFrameDataset(data.Dataset):

    def __init__(self, df, fields, is_test=False, **kwargs):
        examples = []
        for i, row in df.iterrows():
            label = row.target if not is_test else None
            text = row.text
            examples.append(data.Example.fromlist([text, label], fields))

        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex):
        return len(ex.text)

    @classmethod
    def splits(cls, fields, train_df, val_df=None, test_df=None, **kwargs):
        train_data, val_data, test_data = (None, None, None)
        data_field = fields

        if train_df is not None:
            train_data = cls(train_df.copy(), data_field, **kwargs)
        if val_df is not None:
            val_data = cls(val_df.copy(), data_field, **kwargs)
        if test_df is not None:
            test_data = cls(test_df.copy(), data_field, True, **kwargs)

        return tuple(d for d in (train_data, val_data, test_data) if d is not None)

AttributeError: ignored